In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lending-club/rejected_2007_to_2018Q4.csv.gz
/kaggle/input/lending-club/accepted_2007_to_2018Q4.csv.gz
/kaggle/input/lending-club/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv
/kaggle/input/lending-club/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv


In [9]:
# Offline RL pipeline for LendingClub loan approval (Kaggle-ready)
# File: Offline_RL_LendingClub.py
# Notebook-style script. Run on Kaggle (or your local env). Assumes you uploaded the LendingClub CSV as 'lending_club_loan_two.csv'

"""
Overview
--------
This notebook frames loan approval as an *offline reinforcement learning* problem (one-step MDP / contextual bandit):
- State (s): preprocessed applicant features
- Action (a): {0: Deny, 1: Approve}
- Reward (r):
    * If action == 0 (Deny): reward = 0
    * If action == 1 and loan fully paid: reward = +(loan_amnt * int_rate)
    * If action == 1 and loan defaulted: reward = -loan_amnt

We use d3rlpy (a modern offline RL library) and the CQL (Conservative Q-Learning) algorithm.
We estimate the learned policy value with Fitted Q Evaluation (FQE) implemented in d3rlpy.

How to use on Kaggle
---------------------
1. Upload dataset file to the Kaggle notebook (Data pane) and name it `lending_club_loan_two.csv`.
2. Run this notebook. It will: install dependencies, preprocess, create an offline dataset, train CQL, and compute Estimated Policy Value (EPV) using FQE.

Notes
-----
- The script treats each loan instance as a single-step episode (terminal=True).
- The reward engineering follows the user's specification. You may tune reward scaling (e.g., divide by 1e3) to stabilize learning.

"""

# ----------------------
# Install dependencies
# ----------------------
# On Kaggle you can run pip install in a cell. Uncomment the pip install lines if needed.

# !pip install d3rlpy==2.3.1
# !pip install xgboost
# !pip install category_encoders
# !pip install pandas scikit-learn "fsspec>=0.8.4"

"\nOverview\n--------\nThis notebook frames loan approval as an *offline reinforcement learning* problem (one-step MDP / contextual bandit):\n- State (s): preprocessed applicant features\n- Action (a): {0: Deny, 1: Approve}\n- Reward (r):\n    * If action == 0 (Deny): reward = 0\n    * If action == 1 and loan fully paid: reward = +(loan_amnt * int_rate)\n    * If action == 1 and loan defaulted: reward = -loan_amnt\n\nWe use d3rlpy (a modern offline RL library) and the CQL (Conservative Q-Learning) algorithm.\nWe estimate the learned policy value with Fitted Q Evaluation (FQE) implemented in d3rlpy.\n\nHow to use on Kaggle\n---------------------\n1. Upload dataset file to the Kaggle notebook (Data pane) and name it `lending_club_loan_two.csv`.\n2. Run this notebook. It will: install dependencies, preprocess, create an offline dataset, train CQL, and compute Estimated Policy Value (EPV) using FQE.\n\nNotes\n-----\n- The script treats each loan instance as a single-step episode (terminal=

In [16]:
# !pip install d3rlpy==2.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.1/951.1 kB 33.9 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.0
    Uninstalling gymnasium-0.29.0:
      Successfully uninstalled gymnasium-0.29.0
  Attempting uninstall: d3rlpy
    Found existing installation: d3rlpy 2.3.0
    Uninstalling d3rlpy-2.3.0:
      Successfully uninstalled d3rlpy-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 2.1.0 requires gymnasium<0.30,>=0.28.1, but you have gymnasium 1.2.1 which is incompatible.
kaggle-environments 1.18.0 requires gymnasium==0.29.0, but you have gymnasium 1.2.1 which is incompatible.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [17]:
# ----------------------
# Imports
# ----------------------
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import category_encoders as ce

# d3rlpy (offline RL)
import d3rlpy
from d3rlpy.dataset import MDPDataset
from d3rlpy.algos import CQL
from d3rlpy.metrics import evaluate_on_dataset# For FQE
from d3rlpy.dataset import Transition

# ----------------------
# Configuration
# ----------------------
CSV_PATH = '/kaggle/input/lending-club/accepted_2007_to_2018Q4.csv.gz'
# If your CSV is at a different path, update CSV_PATH.

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Reward scaling - recommended to stabilize training. If you want raw currency, set to 1.0
REWARD_SCALE = 1.0  # e.g., 1e-3 to scale dollars to thousands

# Train hyperparams (small by default for Kaggle CPU). Increase epochs on GPU.
N_EPOCHS = 20
BATCH_SIZE = 256

# ----------------------
# Load data
# ----------------------
print('Loading CSV:', CSV_PATH)
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"CSV not found at {CSV_PATH}. Upload the LendingClub CSV to the Kaggle notebook and update CSV_PATH.")

df = pd.read_csv(CSV_PATH, low_memory=False)
print('Raw rows:', len(df))

# ----------------------
# Quick target engineering: map loan_status to binary outcome
# ----------------------
# We'll treat these as 'fully paid' vs 'charged off'/'default' etc.
# Adapt based on your exact CSV's 'loan_status' values. Common values: Fully Paid, Charged Off, Late (31-120 days), Current, etc.

# Keep only accepted statuses for our simple binary outcome

def map_loan_outcome(status):
    s = str(status).lower()
    if 'fully paid' in s:
        return 'fully_paid'
    if 'charged off' in s or 'default' in s:
        return 'defaulted'
    # treat late or other bad states as default for conservatism
    if 'late' in s:
        return 'defaulted'
    # current / in progress - drop these rows because repayment outcome not observed
    return 'other'

if 'loan_status' not in df.columns:
    raise ValueError('CSV does not contain loan_status column. Please check the dataset.')

print('Mapping loan_status to outcomes...')
df['outcome'] = df['loan_status'].apply(map_loan_outcome)
print('Outcome value counts (pre-filter):')
print(df['outcome'].value_counts())

# Drop rows where outcome == 'other' (still in progress)
df = df[df['outcome'].isin(['fully_paid', 'defaulted'])].copy()
print('Rows after dropping in-progress loans:', len(df))

# ----------------------
# Select features
# ----------------------
# Choose a manageable set of features for the example. You can extend this.
FEATURES = [
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
    'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
    'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'revol_util', 'open_acc',
    'pub_rec', 'total_acc'
]

for col in FEATURES:
    if col not in df.columns:
        print(f'Warning: column {col} not found in CSV; it will be skipped')

FEATURES = [c for c in FEATURES if c in df.columns]
print('Using features:', FEATURES)

# Target columns used for reward
if 'loan_amnt' not in df.columns or 'int_rate' not in df.columns:
    raise ValueError('Dataset must contain loan_amnt and int_rate columns for reward calculation.')

ImportError: cannot import name 'evaluate_on_dataset' from 'd3rlpy.metrics' (/usr/local/lib/python3.11/dist-packages/d3rlpy/metrics/__init__.py)

In [ ]:
# ----------------------
# Clean and preprocess
# ----------------------
# Preprocessing pipelines for numeric and categorical
numeric_features = [c for c in FEATURES if df[c].dtype in [np.float64, np.int64] or pd.api.types.is_numeric_dtype(df[c])]
categorical_features = [c for c in FEATURES if c not in numeric_features]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='drop'
)

X_raw = df[FEATURES]
y_raw = df['outcome'].map({'fully_paid': 1, 'defaulted': 0}).astype(int)

print('Preprocessing features...')
X_proc = preprocessor.fit_transform(X_raw)
# category_encoders returns pandas DataFrame for OneHotEncoder; ensure numpy array
if hasattr(X_proc, 'toarray'):
    X_proc = X_proc.toarray()

print('Processed feature shape:', X_proc.shape)

# ----------------------
# Build offline dataset (one-step episodes)
# ----------------------
# Actions: in the dataset we don't have decisions made by a logging policy we can use.
# For offline RL training we need transitions (s,a,r,s',terminal).
# Here we will construct a dataset from the logged decisions by synthetic logging policy:
# Option A: Use the historical decision (if available) — e.g., whether loan was approved by LendingClub.
# Option B: If there's no 'approved' column, we can treat the dataset as full offline context-action-reward by
# generating actions using the historical origination (application accepted) column 'loan_status' or 'is_approved' if present.

# Commonly LendingClub dataset contains 'verification_status' and 'loan_status' but not a direct 'action' column.
# For this pipeline, we'll assume that rows represent *approved loans* (i.e., applicants that were approved and funded),
# and therefore we do not observe rewards for denied actions. This is a classical selection bias issue.
# For demonstration, we'll create a synthetic logging policy: a simple rule-based policy that approves if grade in [A,B,C] and income>some threshold.

# You may replace the synthetic logging policy with the dataset's true origination indicator if available (e.g., 'funded_amnt' > 0)

print('Creating a synthetic logging policy to generate logged actions...')

# Synthetic logging policy (0 deny, 1 approve)
def synthetic_policy_row(row):
    # Approve if grade A/B/C or annual_inc > 50000
    grade = row.get('grade')
    income = row.get('annual_inc', 0)
    if pd.isna(grade):
        return 0
    if grade in ['A', 'B', 'C']:
        return 1
    if income >= 50000:
        return 1
    return 0

# Apply to original df (use raw df to access grade/annual_inc)
df['logged_action'] = df.apply(synthetic_policy_row, axis=1)
print('Logged action distribution (synthetic):')
print(df['logged_action'].value_counts())

# Compute reward per user-specified spec

In [ ]:
def compute_reward(row, reward_scale=REWARD_SCALE):
    a = row['action']
    if a == 0:
        return 0.0
    loan_amnt = float(row['loan_amnt'])
    # int_rate may be like '13.56%' — try to parse
    ir = row['int_rate']
    if isinstance(ir, str) and '%' in ir:
        try:
            ir_val = float(ir.strip().replace('%',''))/100.0
        except:
            ir_val = 0.0
    else:
        ir_val = float(ir)
    if row['outcome'] == 'fully_paid':
        return (loan_amnt * ir_val) / reward_scale
    else:
        return (-loan_amnt) / reward_scale

# For training dataset, we'll use actions from logged_action
train_df = df.copy()
train_df['action'] = train_df['logged_action']
train_df['reward'] = train_df.apply(lambda r: compute_reward(r), axis=1)

# Build arrays for d3rlpy MDPDataset. Since these are one-step, next_state can be zeros and terminal True.
observations = X_proc.astype('float32')
actions = train_df['action'].values.astype('int32')
rewards = train_df['reward'].values.astype('float32')
terminals = np.ones_like(rewards, dtype=bool)  # every transition terminal (single-step episode)
next_observations = np.zeros_like(observations)

print('Observations shape:', observations.shape)
print('Actions shape:', actions.shape)
print('Rewards stats: min %.3f, mean %.3f, max %.3f' % (rewards.min(), rewards.mean(), rewards.max()))

In [ ]:
# Create MDPDataset
mdp_dataset = MDPDataset(observations=observations,
                         actions=actions.reshape(-1,1),
                         rewards=rewards.reshape(-1,1),
                         terminals=terminals,
                         next_observations=next_observations)

# Split train / eval for training the offline RL algorithm and for FQE
obs_train, obs_eval, a_train, a_eval, r_train, r_eval = train_test_split(
    observations, actions, rewards, test_size=0.2, random_state=RANDOM_SEED)

mdp_train = MDPDataset(observations=obs_train,
                       actions=a_train.reshape(-1,1),
                       rewards=r_train.reshape(-1,1),
                       terminals=np.ones_like(r_train, dtype=bool),
                       next_observations=np.zeros_like(obs_train))

mdp_eval = MDPDataset(observations=obs_eval,
                      actions=a_eval.reshape(-1,1),
                      rewards=r_eval.reshape(-1,1),
                      terminals=np.ones_like(r_eval, dtype=bool),
                      next_observations=np.zeros_like(obs_eval))

print('MDP train size:', len(mdp_train.observations))
print('MDP eval size:', len(mdp_eval.observations))

In [ ]:
# ----------------------
# Train an offline RL algorithm: CQL
# ----------------------
# We'll use d3rlpy.CQL

obs_dim = mdp_train.observations.shape[1]
print('Observation dim:', obs_dim)

cql = CQL(
    actor_learning_rate=3e-4,
    critic_learning_rate=3e-4,
    batch_size=BATCH_SIZE,
    n_critics=2,
    # conservative weight controls regularization - tune as necessary
    conservative_weight=5.0,
    q_func_factory=d3rlpy.models.builders.create_q_func,
    scaler='standard'
)

# Fit the model
print('Starting CQL training...')
cql.fit(mdp_train,
        n_epochs=N_EPOCHS,
        eval_episodes=mdp_eval,
        scorers={'average_value': average_value_estimation_scorer},
        verbose=True)

# Save the trained model
MODEL_DIR = '/kaggle/working/cql_model'
os.makedirs(MODEL_DIR, exist_ok=True)
cql.save_model(MODEL_DIR)
print('Saved model to', MODEL_DIR)

# ----------------------
# Policy extraction: deterministic greedy policy from learned Q
# ----------------------
policy = cql.predict_policy

# You can use cql.predict to get actions for observations
sample_obs = mdp_eval.observations[:10]
pred_actions = cql.predict(sample_obs)
print('Sample predicted actions:', pred_actions)

# ----------------------
# Estimate policy value (Estimated Policy Value, EPV) using FQE
# ----------------------
# Fitted Q Evaluation trains a Q-function off-policy and estimates expected return of the evaluation policy.
from d3rlpy.metrics import fqe

print('Running FQE for policy value estimation...')

# Create an evaluation policy wrapper for d3rlpy's functions
class SimplePolicy:
    def __init__(self, model):
        self.model = model
    def predict(self, x: np.ndarray) -> np.ndarray:
        # d3rlpy's algo.predict returns action indices
        return self.model.predict(x.astype('float32'))

eval_policy = SimplePolicy(cql)

# Fit FQE on the training dataset and evaluate the policy
# d3rlpy provides FQE implementation via fqe module

fqe_agent = d3rlpy.algos.FQE(
    batch_size=256,
    n_steps=500,
    gamma=0.99,
    q_func_factory=d3rlpy.models.builders.create_q_func,
    scaler='standard'
)

# Combine train and eval to form an experience replay for FQE
full_mdp = mdp_dataset

# Fit FQE on the logged data
fqe_agent.fit(full_mdp, n_epochs=10)

# Estimate policy value by calling fqe_agent.estimate_return
estimated_return = fqe_agent.estimate_return(full_mdp, eval_policy)
print('Estimated policy value (per-episode return):', estimated_return)

# If you used reward_scale !=1.0, multiply back to get dollar amounts
estimated_return_dollars = estimated_return * REWARD_SCALE
print('Estimated policy value (dollars per decision):', estimated_return_dollars)

# ----------------------
# Logging & checkpoints
# ----------------------
# Save evaluation figures and metrics
metrics_df = pd.DataFrame({
    'estimated_return': [float(estimated_return)],
    'estimated_return_dollars': [float(estimated_return_dollars)],
})
metrics_df.to_csv('/kaggle/working/epv_metrics.csv', index=False)
print('Saved EPV metrics to /kaggle/working/epv_metrics.csv')

print('\nDone. To get final numbers, run this notebook on Kaggle (GPU recommended).')